In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import gc

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-ld_9pw4LMdMZsPPoqWbwUVRyF-KFLcM' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-ld_9pw4LMdMZsPPoqWbwUVRyF-KFLcM" -O resized_data.zip && rm -rf /tmp/cookies.txt

--2023-07-03 15:41:51--  https://docs.google.com/uc?export=download&confirm=&id=1-ld_9pw4LMdMZsPPoqWbwUVRyF-KFLcM
Resolving docs.google.com (docs.google.com)... 172.253.117.139, 172.253.117.138, 172.253.117.101, ...
Connecting to docs.google.com (docs.google.com)|172.253.117.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-8g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sh12bkvp6r76n1qg1pgm1vobrq2tjapk/1688398875000/04268100670184909597/*/1-ld_9pw4LMdMZsPPoqWbwUVRyF-KFLcM?e=download&uuid=79ae46f2-ccc1-43ec-acfd-dad4fab7ee39 [following]
--2023-07-03 15:41:56--  https://doc-14-8g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sh12bkvp6r76n1qg1pgm1vobrq2tjapk/1688398875000/04268100670184909597/*/1-ld_9pw4LMdMZsPPoqWbwUVRyF-KFLcM?e=download&uuid=79ae46f2-ccc1-43ec-acfd-dad4fab7ee39
Resolving doc-14-8g-docs.googleusercontent.com (doc-14-8g-docs.googleusercontent.com)... 172.253.117.1

In [3]:
!rm -r data

rm: cannot remove 'data': No such file or directory


In [4]:
!mkdir data && cd data && unzip -q ../resized_data.zip

In [5]:
train_set = tf.keras.utils.image_dataset_from_directory(
    directory='data/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=((148, 32)),
    validation_split=0.2,
    subset='training',
    seed=123,
    shuffle=True,
)


validation_set = tf.keras.utils.image_dataset_from_directory(
    directory='data/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=((148, 32)),
    validation_split=0.2,
    subset='validation',
    seed=123,
    shuffle=True,
)

Found 1345 files belonging to 4 classes.
Using 1076 files for training.
Found 1345 files belonging to 4 classes.
Using 269 files for validation.


In [6]:
resize_and_rescale = tf.keras.Sequential([
  layers.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  layers.RandomCrop(128, 32),
])

In [7]:
aug_ds = train_set.map(
  lambda x, y: (resize_and_rescale(x, training=True), y))

x_train = aug_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [8]:
for i in range(4):
    y = aug_ds.map(
        lambda x, y: (data_augmentation(x, training=True), y))

    x_train = x_train.concatenate(y)

In [9]:
val_aug_ds = validation_set.map(
  lambda x, y: (resize_and_rescale(x, training=True), y))

val = val_aug_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [10]:
for i in range(4):
    y = val_aug_ds.map(
        lambda x, y: (data_augmentation(x, training=True), y))

    val = val.concatenate(y)

In [13]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(128, 32, 3)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(4, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 30, 64)       1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 63, 15, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 13, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 6, 128)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 4, 64)         73792     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 2, 64)       

In [14]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, batch_size=batch_size, epochs=epochs, validation_data=val)

Epoch 1/15
170/170 [==============================] - 16s 26ms/step - loss: 0.1795 - accuracy: 0.9353 - val_loss: 0.0372 - val_accuracy: 0.9859
Epoch 2/15
170/170 [==============================] - 4s 23ms/step - loss: 0.0694 - accuracy: 0.9777 - val_loss: 0.0243 - val_accuracy: 0.9941
Epoch 3/15
170/170 [==============================] - 5s 30ms/step - loss: 0.0505 - accuracy: 0.9846 - val_loss: 0.0120 - val_accuracy: 0.9970
Epoch 4/15
170/170 [==============================] - 4s 20ms/step - loss: 0.0270 - accuracy: 0.9920 - val_loss: 0.0097 - val_accuracy: 0.9963
Epoch 5/15
170/170 [==============================] - 3s 19ms/step - loss: 0.0347 - accuracy: 0.9894 - val_loss: 0.0069 - val_accuracy: 0.9970
Epoch 6/15
170/170 [==============================] - 5s 27ms/step - loss: 0.0299 - accuracy: 0.9929 - val_loss: 0.0047 - val_accuracy: 0.9985
Epoch 7/15
170/170 [==============================] - 3s 20ms/step - loss: 0.0212 - accuracy: 0.9928 - val_loss: 0.0207 - val_accuracy: 0.991

In [15]:
model.save('iot_model.h5')